In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

### 0. Understanding the Business Problem
Uber Inc in the US wants to know:

- the major complaints premium users have about their cab services,
- how these impact service ratings.

We as (technical) consultants to Uber. have to:  
- [a] analyze text reviews of Uber cabs’ US services,  
- [b] relate whether and which different features of these reviews impact overall ratings  
- [c] pinpoint possible areas of improvement.

### 1. Pre-processing: 
- Examine the dataset. 
- ID the columns of interest. 
- Drop special characters, html junk etc. 
- Perform any other preprocessing and text-cleaning activity you think fits this context.

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/Kenrich005/Uber_reviews_textanalytics/main/uber_reviews_itune.csv",
                 encoding='cp1252')
df.head()

,Author_Name,Title,Author_URL,App_Version,Rating,Review,Date
0,#NEVERUBER,Dishonest and Disgusting,https://itunes.apple.com/us/reviews/id663331949,3.434.10005,1,"For half an hour, we tried EVERY UBER SERVICE ...",29-12-2020 01:14
1,$$Heaven,Free offer,https://itunes.apple.com/us/reviews/id810421958,3.434.10005,2,If I’m not eligible for the offer Stop floodin...,01-01-2021 23:17
2,.Disappointed....,Inaccurate,https://itunes.apple.com/us/reviews/id49598333,3.439.10000,2,Consistently inaccurate Uber Eats ETA and the ...,15-01-2021 23:38
3,.i. andrea,bad,https://itunes.apple.com/us/reviews/id689880334,3.434.10005,1,i had my rides canceled back to back. they the...,08-12-2020 01:01
4,-:deka:-,Double charged me for an order,https://itunes.apple.com/us/reviews/id124963835,3.434.10005,1,Two of the same orders was added by accident. ...,15-12-2020 04:02


Columns of interest:  
1. Title - Brief summary about the review
2. Rating - Label for supervised learning
3. Review - To extract the sentiment of the complaint
4. Date - Extracting weekday or weekend may give better insight on nature of review

### Data Cleaning

In [3]:
df1 = df.drop(['Author_Name','Author_URL','App_Version'],axis=1)
df1.head()

,Title,Rating,Review,Date
0,Dishonest and Disgusting,1,"For half an hour, we tried EVERY UBER SERVICE ...",29-12-2020 01:14
1,Free offer,2,If I’m not eligible for the offer Stop floodin...,01-01-2021 23:17
2,Inaccurate,2,Consistently inaccurate Uber Eats ETA and the ...,15-01-2021 23:38
3,bad,1,i had my rides canceled back to back. they the...,08-12-2020 01:01
4,Double charged me for an order,1,Two of the same orders was added by accident. ...,15-12-2020 04:02


In [4]:
# Replacing emoticon with its respective meaning
df_emojis = pd.read_csv("https://raw.githubusercontent.com/Kenrich005/Uber_reviews_textanalytics/main/emoji_description.csv")
df_emojis['Code'] = df_emojis['Code'].str.replace('+','+000')
df_emojis.head()

,Code,CLDR Short Name
0,<U+0001F600>,grinning face
1,<U+0001F603>,grinning face with big eyes
2,<U+0001F604>,grinning face with smiling eyes
3,<U+0001F601>,beaming face with smiling eyes
4,<U+0001F606>,grinning squinting face


In [5]:
# Replacing emoticon with its respective meaning
to_replace = df_emojis.Code.tolist()
replace_with = df_emojis['CLDR Short Name'].tolist()

# using zip() to convert lists to dictionary
res = dict(zip(to_replace, replace_with))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [6]:
df1.Review = df1.Review.apply(lambda text: replace_all(text, res))
df1.Review[1]

'If I’m not eligible for the offer Stop flooding my email with this false information pouting facepouting facepouting face'

In [7]:
df1.Review = df1.Review.str.split('<').str[0]
df1.shape

(490, 4)

In [8]:
df1.Review[1]

'If I’m not eligible for the offer Stop flooding my email with this false information pouting facepouting facepouting face'

In [9]:
df1['Review'].replace('', np.nan, inplace=True)
df1.dropna(subset=['Review'], inplace=True)
df1.shape

(489, 4)

In [10]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# define unit func to process one doc
from nltk import sent_tokenize, word_tokenize
def vader_unit_func(doc0,column_name):
    sents_list0 = sent_tokenize(doc0)
    vs_doc0 = []
    sent_ind = []
    for i in range(len(sents_list0)):
        vs_sent0 = analyzer.polarity_scores(sents_list0[i])
        vs_doc0.append(vs_sent0)
        sent_ind.append(i)
        
    # obtain output as DF    
    doc0_df = pd.DataFrame(vs_doc0)
    doc0_df.columns = [x+column_name for x in doc0_df.columns]
    doc0_df.insert(0, 'sent_index', sent_ind)  # insert sent index
    doc0_df.insert(doc0_df.shape[1], 'sentence', sents_list0)
    return(doc0_df)

# define wrapper func
def vader_wrap_func(corpus0,column_name):
    
    # use ifinstance() to check & convert input to DF
    if isinstance(corpus0, list):
        corpus0 = pd.DataFrame({'text':corpus0})
    
    # define empty DF to concat unit func output to
    vs_df = pd.DataFrame()    
    
    # apply unit-func to each doc & loop over all docs
    for i1 in range(len(corpus0)):
        doc0 = str(corpus0.iloc[i1])
        vs_doc_df = vader_unit_func(doc0,column_name)  # applying unit-func
        vs_doc_df.insert(0, 'doc_index', i1)  # inserting doc index
        vs_df = pd.concat([vs_df, vs_doc_df], axis=0)
        
    return(vs_df)

In [11]:
# test-drive wrapper func
review_sentiment = vader_wrap_func(df1.Review,'_review').groupby('doc_index').sum()
title_sentiment = vader_wrap_func(df1.Title,'_title').groupby('doc_index').sum()
df1 = pd.concat([df1,review_sentiment,title_sentiment],axis=1)
print(df1.shape)
df1.head()

(490, 14)


,Title,Rating,Review,Date,sent_index,neg_review,neu_review,pos_review,compound_review,sent_index,neg_title,neu_title,pos_title,compound_title
0,Dishonest and Disgusting,1.0,"For half an hour, we tried EVERY UBER SERVICE ...",29-12-2020 01:14,3.0,0.000,2.876,0.124,0.1406,0.0,0.877,0.123,0.000,-0.7964
1,Free offer,2.0,If I’m not eligible for the offer Stop floodin...,01-01-2021 23:17,0.0,0.109,0.891,0.000,-0.2960,0.0,0.000,0.233,0.767,0.5106
2,Inaccurate,2.0,Consistently inaccurate Uber Eats ETA and the ...,15-01-2021 23:38,0.0,0.179,0.821,0.000,-0.3400,0.0,0.000,1.000,0.000,0.0000
3,bad,1.0,i had my rides canceled back to back. they the...,08-12-2020 01:01,10.0,1.167,3.592,0.241,-0.1617,0.0,1.000,0.000,0.000,-0.5423
4,Double charged me for an order,1.0,Two of the same orders was added by accident. ...,15-12-2020 04:02,21.0,0.908,5.614,0.478,-0.4906,0.0,0.265,0.735,0.000,-0.2023


In [12]:
# Converting Date into datetime format
df1['Date'] =  pd.to_datetime(df1['Date'], format='%d-%m-%Y %H:%M')
df1.Date.head()

0   2020-12-29 01:14:00
1   2021-01-01 23:17:00
2   2021-01-15 23:38:00
3   2020-12-08 01:01:00
4   2020-12-15 04:02:00
Name: Date, dtype: datetime64[ns]

In [13]:
df1['Isweekend'] = np.where(df1.Date.dt.dayofweek>4,1,0)
df1['Late_night'] = np.where(df1.Date.dt.hour<4,1,0)
df1['Early_mrng'] = np.where(df1.Date.dt.hour.between(4,8),1,0)
df1['Morning'] = np.where(df1.Date.dt.hour.between(8,12),1,0)
df1['Noon'] = np.where(df1.Date.dt.hour.between(12,16),1,0)
df1['Eve'] = np.where(df1.Date.dt.hour.between(16,20),1,0)
df1['Night'] = np.where(df1.Date.dt.hour>20,1,0)
df1.head()

,Title,Rating,Review,Date,sent_index,neg_review,neu_review,pos_review,compound_review,sent_index,...,neu_title,pos_title,compound_title,Isweekend,Late_night,Early_mrng,Morning,Noon,Eve,Night
0,Dishonest and Disgusting,1.0,"For half an hour, we tried EVERY UBER SERVICE ...",2020-12-29 01:14:00,3.0,0.000,2.876,0.124,0.1406,0.0,...,0.123,0.000,-0.7964,0,1,0,0,0,0,0
1,Free offer,2.0,If I’m not eligible for the offer Stop floodin...,2021-01-01 23:17:00,0.0,0.109,0.891,0.000,-0.2960,0.0,...,0.233,0.767,0.5106,0,0,0,0,0,0,1
2,Inaccurate,2.0,Consistently inaccurate Uber Eats ETA and the ...,2021-01-15 23:38:00,0.0,0.179,0.821,0.000,-0.3400,0.0,...,1.000,0.000,0.0000,0,0,0,0,0,0,1
3,bad,1.0,i had my rides canceled back to back. they the...,2020-12-08 01:01:00,10.0,1.167,3.592,0.241,-0.1617,0.0,...,0.000,0.000,-0.5423,0,1,0,0,0,0,0
4,Double charged me for an order,1.0,Two of the same orders was added by accident. ...,2020-12-15 04:02:00,21.0,0.908,5.614,0.478,-0.4906,0.0,...,0.735,0.000,-0.2023,0,0,1,0,0,0,0


In [14]:
df1=df1.drop(['sent_index','Title','Review','Date'],axis=1)
df1.head()

,Rating,neg_review,neu_review,pos_review,compound_review,neg_title,neu_title,pos_title,compound_title,Isweekend,Late_night,Early_mrng,Morning,Noon,Eve,Night
0,1.0,0.000,2.876,0.124,0.1406,0.877,0.123,0.000,-0.7964,0,1,0,0,0,0,0
1,2.0,0.109,0.891,0.000,-0.2960,0.000,0.233,0.767,0.5106,0,0,0,0,0,0,1
2,2.0,0.179,0.821,0.000,-0.3400,0.000,1.000,0.000,0.0000,0,0,0,0,0,0,1
3,1.0,1.167,3.592,0.241,-0.1617,1.000,0.000,0.000,-0.5423,0,1,0,0,0,0,0
4,1.0,0.908,5.614,0.478,-0.4906,0.265,0.735,0.000,-0.2023,0,0,1,0,0,0,0


In [15]:
# Removing null values
df1.dropna(inplace=True)

### Preliminary Regression Model


In [16]:
y = df1.Rating
X = df1.drop('Rating', axis=1)
y.shape, X.shape

((488,), (488, 15))

In [17]:
X.isnull().sum()

neg_review         0
neu_review         0
pos_review         0
compound_review    0
neg_title          0
neu_title          0
pos_title          0
compound_title     0
Isweekend          0
Late_night         0
Early_mrng         0
Morning            0
Noon               0
Eve                0
Night              0
dtype: int64

In [18]:
import statsmodels.api as sm
X = sm.add_constant(X)
model = sm.OLS(y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Rating   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9251
Date:                Mon, 11 Jul 2022   Prob (F-statistic):              0.536
Time:                        23:50:22   Log-Likelihood:                -690.32
No. Observations:                 488   AIC:                             1413.
Df Residuals:                     472   BIC:                             1480.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               1.5702      0.312      5.030      0.000       0.957       2.183
neg_review         -0.0286      0.170     -0.168      0.866      -0.362       0.305
neu_review         -0.0176      0.020     -0.868      0.386      -0.057       0.022
pos_review          0.1158      0.204      0.568      0.570      -0.285       0.516
compound_review     0.0083      0.103      0.081      0.936      -0.194       0.210
neg_title           0.0246      0.387      0.063      0.949      -0.737       0.786
neu_title          -0.0241      0.232     -0.104      0.917      -0.479       0.431
pos_title           0.0220      0.413      0.053      0.958      -0.790       0.834
compound_title      0.2870      0.365      0.787      0.432      -0.430       1.004
Isweekend          -0.1134      0.102     -1.111      0.267      -0.314       0.087
Late_night          0.1770      0.238      0.743      0.458      -0.291       0.645
Early_mrng         -0.1633      0.230     -0.710      0.478      -0.615       0.289
Morning            -0.1284      0.199     -0.646      0.518      -0.519       0.262
Noon               -0.0113      0.165     -0.068      0.945      -0.335       0.312
Eve                 0.1058      0.210      0.505      0.614      -0.306       0.518
Night               0.1374      0.237      0.579      0.563      -0.329       0.604
==============================================================================
Omnibus:                      205.728   Durbin-Watson:                   2.117
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              631.643
Skew:                           2.078   Prob(JB):                    6.93e-138
Kurtosis:                       6.714   Cond. No.                         52.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
X.drop('const',axis=1,inplace=True)
X.head()

,neg_review,neu_review,pos_review,compound_review,neg_title,neu_title,pos_title,compound_title,Isweekend,Late_night,Early_mrng,Morning,Noon,Eve,Night
0,0.000,2.876,0.124,0.1406,0.877,0.123,0.000,-0.7964,0,1,0,0,0,0,0
1,0.109,0.891,0.000,-0.2960,0.000,0.233,0.767,0.5106,0,0,0,0,0,0,1
2,0.179,0.821,0.000,-0.3400,0.000,1.000,0.000,0.0000,0,0,0,0,0,0,1
3,1.167,3.592,0.241,-0.1617,1.000,0.000,0.000,-0.5423,0,1,0,0,0,0,0
4,0.908,5.614,0.478,-0.4906,0.265,0.735,0.000,-0.2023,0,0,1,0,0,0,0


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((341, 15), (147, 15), (341,), (147,))

### Calculating VIF

In [21]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):
   
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'].sort_values()

    return(vif)

In [22]:
calc_vif(X)

,variables,VIF
0,neg_review,4.793164
1,neu_review,3.537584
2,pos_review,4.679958
3,compound_review,3.464564
4,neg_title,8.899032
5,neu_title,8.529243
6,pos_title,3.213655
7,compound_title,7.530797
8,Isweekend,1.469245
9,Late_night,3.464104


Generally, a VIF above 4 or tolerance below 0.25 indicates that multicollinearity might exist, and further investigation is required.   
When VIF is higher than 10 or tolerance is lower than 0.1, there is significant multicollinearity that needs to be corrected.  
  
Since all the above variables have VIF below 4 and above 0.25, we can be assured that there is no multicollinearity.

In [23]:
# We will save the model performance metrics in a DataFrame

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score
Model = []
RMSE = []
R_sq = []
cv = KFold(5)

#Creating a Function to append the cross validation scores of the algorithms
def input_scores(name, model, x, y):
    Model.append(name)
    RMSE.append(np.sqrt((-1) * cross_val_score(model, x, y, cv=cv, scoring='neg_mean_squared_error').mean()))
    R_sq.append(cross_val_score(model, x, y, cv=cv, scoring='r2').mean())

In [24]:
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor)

names = ['Linear Regression', 'Ridge Regression', 'Lasso Regression',
         'K Neighbors Regressor', 'Decision Tree Regressor', 
         'Random Forest Regressor', 'Gradient Boosting Regressor',
         'Adaboost Regressor','XGBRegressor']

models = [LinearRegression(), Ridge(), Lasso(),
          KNeighborsRegressor(), DecisionTreeRegressor(),
          RandomForestRegressor(), GradientBoostingRegressor(), 
          AdaBoostRegressor(),XGBRegressor()]

#Running all algorithms
for name, model in zip(names, models):
    input_scores(name, model, X_train, y_train)

Reference: https://www.kaggle.com/swatisinghalmav/best-of-8-regression-models-to-predict-strength

In [25]:
evaluation = pd.DataFrame({'Model': Model,'RMSE': RMSE,'R Squared': R_sq})
print("FOLLOWING ARE THE TRAINING SCORES: ")
evaluation

FOLLOWING ARE THE TRAINING SCORES: 


,Model,RMSE,R Squared
0,Linear Regression,1.059369,-0.110406
1,Ridge Regression,1.052327,-0.093232
2,Lasso Regression,1.013998,-0.009029
3,K Neighbors Regressor,1.146814,-0.317307
4,Decision Tree Regressor,1.395793,-1.092606
5,Random Forest Regressor,1.117822,-0.229561
6,Gradient Boosting Regressor,1.162286,-0.346336
7,Adaboost Regressor,1.173657,-0.335848
8,XGBRegressor,1.209592,-0.459199


## Next Steps:
1. Convert non-English reviews to English or use non-english dictionary - ## need to add to current code
2. Scale the emoticons replacement - ## validation pending
3. Make sentiment analysis of Title - ## Done
4. From Date, extract weekend, weekday, morning, afternoon, evening, night - ## Done
5. Make preliminary regression model with y variable as Ratings - ## Done
6. ?Use OLS - ## Done
7. Feature Engineering - columns on specific word count
8. Shiny App
9. Add PCA